In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML
import warnings
warnings.filterwarnings('ignore')

# Exercise 10.15
Recall that the trapezoidal method is defined by

\begin{align*}
u(t + \Delta t) \approx u(t) + \frac{\Delta t}{2}\left[f(u(t)) + f(u(t + \Delta t)) \right]
\end{align*}

but remember our ode we are trying to solve is $u' = \lambda u$ meaning $f(u(t)) = \lambda u(t)$.
Even though the trapezoidal method is an implicit method we can still solve explicitly for $u(t + \Delta t)$ because our
vector field is linear.

\begin{align*}
u(t + \Delta t) &\approx u(t) + \frac{\Delta t}{2}\left[\lambda u(t) + \lambda u(t + \Delta t) \right]\\
u(t + \Delta t) \left[1  - \frac{\Delta t \lambda}{2}\right] & \approx u(t)\left[1 + \frac{\Delta t \lambda}{2}\right]\\
u(t + \Delta t) &\approx \frac{u(t)\left[1 + \frac{\Delta t \lambda}{2} \right]}{1 - \frac{\Delta t \lambda}{2} }\\
u(t + \Delta t) &\approx u(t) + \left[\frac{2 + \Delta t \lambda}{2 - \Delta t \lambda} \right]
\end{align*}

Remember that $\lambda_k = - aik - k^2$ where $k$ is the wave number.
This leads to the following algorithm:

In [2]:
def advec_diffuse_trap(a,dt,T,x=[],u0=[]):
    """Uses a spectral decomposition and forward Euler to solve
        the 1D advection diffusion equation u_t + au_x = u_xx.
        Assumes the domain is [0,2pi] or else the wavenumbers would
        need to be weighted differently.
        a is the advection speed
        dt is the time-step
        T is the final time we run to
        x is the spatial discretization
        u0 is the initial condition
        Because this uses the fft, it is best to have len(x) = 2^K
        or some integer K
    """

    N = len(x)
    #The most difficult part is
    #figuring out how numpy indexes the wavenumbers
    k = np.array([1j*y for y in range(0,int(N/2))] + [0] + [1j*y for y in range(int(-N/2)+1,0)])
    k2 = k**2;
    u = []
    u.append(u0)
    uhat = np.fft.fft(u0)
    totalStep = int(T/dt)
    for nn in range(totalStep):
        # trapezoid time-stepping
        new_uhat = uhat * (2 + dt *(-a*k + k2)) / (2 - dt*(-a*k + k2))
        uhat = new_uhat.copy()
        # revert back to real values (not necessary unless we want
        # to save our solution in real space)
        u.append(np.real(np.fft.ifft(uhat)))
    return x, np.array(u), dt

def problem10_15_vals():
    N = 128
    x = np.linspace( 0, 2*np.pi, N )
    a = 4
    T= 0.5
    dt = 1/250
    u0 = np.sin( x**2 ) + np.cos( 5 * x )
    
    return advec_diffuse_trap( a, dt, T, x, u0)

#get values
x, u, dt = problem10_15_vals()
#creae animation
plt.ioff()
fig = plt.figure()
fig.set_dpi(150)
ax = fig.add_subplot(111)
ax.set_xlim(0, 2*np.pi)
plt.suptitle(r'Advection Diffusion Equation $\Delta t = $' + str(dt))
#update function
def update(i):
    ax.clear()
    ax.set_ylim(-2.1, 2.1)
    ax.plot(x, u[i])
    ax.set_xlabel('x')
    ax.set_ylabel('u')
    return ax

ani = animation.FuncAnimation(fig, update,
                              frames = range(u.shape[0]), 
                              interval=35)
HTML(ani.to_html5_video())

# Exercise 10.16

We now derive the forward Euler discretization of the Kuramato-Sivashinsky (KS) equation on $[0, 2\pi]$:
\begin{align*}
u_t + uu_{x} + u_{xx} + u_{xxxx} &=0
\end{align*}
Notice that this is a quasi-linear PDE so we will use eigenfunction expansions for quasi-linear PDEs and a numerical pseudo-spectral method. We first consider the ansatz $u(x, t) = \sum_{k} \widehat{u}_k(t) e^{ikx}$
plugging this into our PDE we get

\begin{align*}
\sum_{k} \widehat{u}_k'(t) e^{ikx} + \sum_{k} \widehat{u}_k(t)e^{ikx}\sum_{n} i n \widehat{u}_n(t) e^{inx} -k^2 \sum_{k}\widehat{u}_k(t) e^{ikx} + k^4 \sum_{k}k^4 \widehat{u}_k(t) e^{ikx} &=0
\end{align*}

Recall that $e^{ikx}$ is an orthogonal basis of $L^2(\Omega)$ (we proved this in volume 2) using this we get

\begin{align*}
\sum_{k} \widehat{u}_k'(t) \int_0^{2\pi} e^{ikx} e^{imx} + \sum_{k, n} \widehat{u}_k(t) \cdot in \widehat{u}_n(t) \int_0^{2\pi} e^{ikx} e^{inx} e^{imx} dx -k^2 \sum_{k}\widehat{u}_k(t)\int_0^{2\pi} e^{ikx} e^{imx} dx + k^4 \sum_{k}k^4 \widehat{u}_k(t) \int_0^{2\pi} e^{ikx}e^{imx}dx &=0
\end{align*}

And notice that

\begin{align*}
    \int_0^{2\pi} e^{ikx} e^{inx} e^{imx} dx = \frac{-i\left(-1 + e^{2i\pi (k +m +n)} \right)}{k+m+n}= \frac{-i\left(-1 + \cos(2\pi(k+m+n)) +i \sin(2\pi (k+m+n)) \right)}{k+m+n} = 0 \textrm{ if } n\neq m\neq k
\end{align*}

Which allows us to give our formula as

\begin{align*}
    \widehat{u}_t + \widehat{u}\cdot ik \widehat{u} - k^2 \widehat{u} + k^4 \widehat{u} &= 0\\
    \widehat{u}_t + \mathscr{F}\left( \mathscr{F}^{-1}(\widehat{u}) \cdot \mathscr{F}^{-1}(ik\widehat{u})\right) + (k^4 - k^2)\widehat{u} &=0
\end{align*}

Inserting our forward Euler method leads to 

\begin{align*}
\frac{\widehat{u}(t + \Delta t) - \widehat{u}(t))}{\Delta t} + \mathscr{F}\left( \mathscr{F}^{-1}(\widehat{u}(t)) \cdot \mathscr{F}^{-1}(ik\widehat{u}(t))\right) + (k^4 - k^2)\widehat{u}(t) = 0\\
\widehat{u}(t + \Delta t) = \widehat{u}(t) + \Delta t\left[ (k^2 - k^4) \widehat{u}(t) -\mathscr{F}\left( \mathscr{F}^{-1}(\widehat{u}(t)) \cdot \mathscr{F}^{-1}(ik\widehat{u}(t))\right) \right]
\end{align*}

which is our update function. This is expressed in the following algorithm (note that $i$ is included in the wave numbers in this algorithm so we need to switch the sign in from of the variable $k^2$ to account for this change):

In [4]:
def diffusive_burgers(dt,T,x=[],u0=[]):
    """Uses a spectral decomposition and forward Euler to solve
        the 1D diffusive Burgers equation u_t + uu_x = u_xx. Assumes
        the domain is [0,2pi] or else the wavenumbers would need to
        be weighted differently. 
        dt is the time-step 
        T is the final time we run to 
        x is the spatial discretization u0 is the
        initial condition Because this uses the fft, 
        it is best to have len(x) = 2^K - for some integer K
    """
    N = len(x)
    #The most difficult part is figuring out how numpy indexes the wavenumbers
    k = np.array([y*1j for y in range(0,int(N/2))] + [0] + [y*1j for y in range(int(-N/2)+1,0)])
    k2 = k**2;
    k4 = k**4
    u = []
    u.append(u0)
    uhat = np.fft.fft(u0)
    totalStep = int(T/dt)
    for nn in range(totalStep):
        # forward Euler time-stepping
        new_uhat = uhat + dt*(-np.fft.fft(np.fft.ifft(k*uhat)*np.fft.ifft(uhat)) + (-k2 - k4)*uhat )
        uhat = new_uhat.copy()
        # revert back to real values (not necessary unless we want
        # to save our solution in real space)
        u.append(np.real(np.fft.ifft(uhat)))
    return x,np.array(u), dt